In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings

from collections import Counter
warnings.filterwarnings('ignore')

In [3]:
# данные по магазинам
st_df = pd.read_csv(r'/content/drive/MyDrive/sp_sales_task/st_df.csv')

# данные по иерархии товаров
pr_df = pd.read_csv(r'/content/drive/MyDrive/sp_sales_task/pr_df.csv')

# данные по продажам
sales_df_train = pd.read_csv(r'/content/drive/MyDrive/sp_sales_task/sales_df_train.csv')

# календарь
calendar = pd.read_csv(r'/content/drive/MyDrive/sp_sales_task/holidays_covid_calendar.csv')

In [4]:
# мерджим флаг активности
sales_df_train = pd.merge(sales_df_train, st_df[['st_id','st_is_active']], on='st_id', how='left')

In [5]:
# выбираем только рабочие магазины
sales_df_train = sales_df_train[sales_df_train['st_is_active'] == 1]

In [6]:
# избавляемся от возвратных позиций
sales_df_train = sales_df_train[sales_df_train['pr_sales_in_rub'] >= 0]

In [7]:
# информация по размеру дф
sales_df_train.shape

(882208, 9)

- [x] Удалить продажи более 15к
- [x] Удалить строки, где объем = 0
- [x] Посмотреть по магазинам, если они продают мало, то тоже удалить

In [8]:
# создал переменную, куда скопировал основной дф
sales_df_train_1 = sales_df_train

In [9]:
# удаляем все строки, где объем продаж = 0
sales_df_train_1 = sales_df_train_1[sales_df_train_1['pr_sales_in_units'] != 0]

In [10]:
# информация по размеру дф
sales_df_train_1.shape

(815674, 9)

In [11]:
# удаляю строки, где сумма продаж по 1 позиции меньше 15 тыс. руб.
sales_df_train_1 = sales_df_train_1[sales_df_train_1['pr_sales_in_rub'] < 15000]

In [12]:
# информация по размеру дф
sales_df_train_1.shape

(814100, 9)

In [13]:
# агрегирую продажи по магазинам
gr_shop = sales_df_train_1.groupby('st_id').agg({'pr_sales_in_rub':'sum', 'pr_sales_in_units':'sum'}).reset_index()

In [14]:
# информация по суммарным продажам по магазинам
gr_shop.sort_values(by='pr_sales_in_units')

,st_id,pr_sales_in_rub,pr_sales_in_units
0,084a8a9aa8cced9175bd07bc44998e75,6836.0,49.0
6,bd470ca955d9497bbcb808e59952fffc,251196.0,5114.0
4,53e19f3dbb211f20b20b45668303c1b6,1072659.0,19227.0
2,1ecfb463472ec9115b10c292ef8bc986,8258219.0,129190.0
9,fa7cdfad1a5aaf8370ebeda47a1ff1c3,60084254.0,496844.0
5,6364d3f0f495b6ab9dcf8d3b5c6e0b01,61109229.0,498561.0
8,f7e6c85504ce6e82442c770f7c8606f0,70936095.0,638501.0
3,42a0e188f5033bc65bf8d78622277c4e,82711537.0,683572.0
1,16a5cdae362b8d27a1d8f8c7b78b4330,93872972.0,819797.0
7,c81e728d9d4c2f636f067f89cc14862c,106060125.0,938747.0


In [15]:
# удаляю первые три магазина, где меньше всего продаж
sales_df_train_1 = sales_df_train_1[~sales_df_train_1['st_id'].isin(['084a8a9aa8cced9175bd07bc44998e75',
                                                                  'bd470ca955d9497bbcb808e59952fffc',
                                                                  '53e19f3dbb211f20b20b45668303c1b6'])]

In [16]:
sales_df_train_1.shape

(802545, 9)

In [17]:
print(round(100 - (802545 / 882208 * 100), 2), '% удаленных данных')

9.03 % удаленных данных


In [18]:
# находим стоимость за единицу продукции
sales_df_train_1['price'] = sales_df_train_1['pr_sales_in_rub'] / sales_df_train_1['pr_sales_in_units']

In [19]:
# вывожу строки, где цена = 0
sales_df_train_1[sales_df_train_1['price'] == 0]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_is_active,price
30,f7e6c85504ce6e82442c770f7c8606f0,3474f0e025c7deb7e127a35763bc5512,2023-06-06,1,8.0,8.0,0.0,0.0,1,0.0
7432,c81e728d9d4c2f636f067f89cc14862c,90252502ab29331e94fd3113f11c58e0,2022-12-27,1,2.0,2.0,0.0,0.0,1,0.0
8153,f7e6c85504ce6e82442c770f7c8606f0,4e3d5c9fab3fc51a4857846fea3d2e57,2022-12-19,1,1.0,1.0,0.0,0.0,1,0.0
8931,f7e6c85504ce6e82442c770f7c8606f0,0f152427918d29bb1081834c1d375a48,2022-10-28,1,2.0,2.0,0.0,0.0,1,0.0
10269,f7e6c85504ce6e82442c770f7c8606f0,465e4f9b8d22467e46a6e02a78a40e97,2023-01-17,0,1.0,0.0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...
876801,1ecfb463472ec9115b10c292ef8bc986,525526d5b2169462195c44d49fff4a44,2022-10-22,1,1.0,1.0,0.0,0.0,1,0.0
878163,fa7cdfad1a5aaf8370ebeda47a1ff1c3,27e8978c9e8b7146c462feb7d24f5637,2022-10-27,1,9.0,9.0,0.0,0.0,1,0.0
882360,6364d3f0f495b6ab9dcf8d3b5c6e0b01,2dc922d957188fb51dd2bd50ec9b555a,2022-10-29,1,12.0,12.0,0.0,0.0,1,0.0
882380,f7e6c85504ce6e82442c770f7c8606f0,9da31b9d3cf407cd724c65b7dfed00ee,2022-11-27,1,3.0,3.0,0.0,0.0,1,0.0


In [20]:
# удаляю строки, где price = 0
sales_df_train_1.drop(sales_df_train_1[sales_df_train_1['price'] == 0].index, inplace=True)

In [21]:
sales_df_train_1.shape

(801778, 10)

In [22]:
# небольшие махинации для дальнейшего метода merge
sales_df_train_1.drop(columns='st_is_active', inplace=True)
calendar['calday'] = calendar['calday'].astype(str)
sales_df_train_1['date'] = pd.to_datetime(sales_df_train_1['date']).dt.strftime('%Y%m%d')

In [23]:
sales_df_train_1 = pd.merge(sales_df_train_1, calendar, left_on='date',
                          right_on='calday', how='left')

In [24]:
# приводим в приличный вид
sales_df_train_1.drop(columns='date_y', inplace=True)
sales_df_train_1.rename(columns={'date_x':'date'}, inplace=True)

In [25]:
# добавляем фич с номером месяца
sales_df_train_1['month'] = sales_df_train_1['calday'].str[4:6]

In [26]:
# добавляем фичи с товарной иерархией
sales_df_train_1 = pd.merge(sales_df_train_1, pr_df, on='pr_sku_id', how='left')

In [27]:
# добавляем фичи с данными по магазинам
sales_df_train_1 = pd.merge(sales_df_train_1, st_df, on='st_id', how='left')

In [28]:
sales_df_train_1

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,price,year,...,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active
0,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,20221020,1,5.0,5.0,825.0,825.0,165.00,2022,...,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1
1,42a0e188f5033bc65bf8d78622277c4e,68f441429e73dbd33b81f95da31576e9,20230129,0,4.0,0.0,196.0,0.0,49.00,2023,...,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,68f204f3838bfee4ada868b66e6a0814,17,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1
2,1ecfb463472ec9115b10c292ef8bc986,22988026fe85fde60d8bf777ab6bc3b4,20230302,0,1.0,0.0,78.0,0.0,78.00,2023,...,c74d97b01eae257e44aa9d5bade97baf,f3173935ed8ac4bf073c1bcd63171f8a,5fd72b9fc7118bd36a137d7cea8c588a,17,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,2,1,28,1
3,16a5cdae362b8d27a1d8f8c7b78b4330,e6fc24ca846339d204b0b50e9f772484,20230217,0,1.0,0.0,161.0,0.0,161.00,2023,...,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,0d51481bbf77027ddb2c990cf97cd9dd,1,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1
4,42a0e188f5033bc65bf8d78622277c4e,3474f0e025c7deb7e127a35763bc5512,20221103,0,3.0,0.0,123.0,0.0,41.00,2022,...,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,ca34f669ae367c87f0e75dcae0f61ee5,17,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801773,c81e728d9d4c2f636f067f89cc14862c,a0150fefa71d6606d5e642fa0e7b8ad8,20221213,1,1.0,1.0,33.0,33.0,33.00,2022,...,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,8ec316a6bd7240d74c0567a1d9b311e5,17,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1
801774,42a0e188f5033bc65bf8d78622277c4e,5b146d93f822459260216df28e572eca,20230418,0,1.0,0.0,192.0,0.0,192.00,2023,...,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,6bae9a61fa499b24f28c3ddcef6ccb1b,1,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1
801775,42a0e188f5033bc65bf8d78622277c4e,c8bd18ea08d2f47cb0a7992f11ada9de,20220816,0,5.0,0.0,970.0,0.0,194.00,2022,...,aab3238922bcc25a6f606eb525ffdc56,186a157b2992e7daed3677ce8e9fe40f,aa36dc1d45661519a5bf923429776a83,1,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1
801776,f7e6c85504ce6e82442c770f7c8606f0,b960c6ee5b5efa12e73f810204b561a8,20230423,0,4.0,0.0,262.0,0.0,65.50,2023,...,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,8ec316a6bd7240d74c0567a1d9b311e5,17,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,1,1,12,1
